In [ ]:
# 알집 푸는 코드
!unzip CNRPark-Patches-150x150-20210107T023432Z-001.zip -d CNRPark-Patches-150x150

In [ ]:
import cv2
import numpy as np
import glob
import os
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import Dataset

BATCH_SIZE = 64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 학습을 위한 모델 설계

In [ ]:
class UltraSimpleParkingSpaceClassificationNet(nn.Module):
  def __init__(self):
    super(UltraSimpleParkingSpaceClassificationNet, self).__init__()
    self.layers = nn.Sequential(      # convolution model
        nn.Conv2d(3, 32, 3 ,padding=1),   # input_channel:3, output_channel:32, kernel의 크기: 3
        nn.ReLU()
    )   

    self.fc_layers = nn.Sequential(
        nn.Linear(150 * 150 * 32, 100),
        nn.ReLU(),
        nn.Linear(100, 2)
    )

  def forward(self, x):
    out = self.layers(x)
    out = out.view(BATCH_SIZE, -1)
    out = self.fc_layers(out)
    return out

In [ ]:
class ImageMultiFolder(Dataset):
  def __init__(self, data_root):
    self.imgNames = []

    for parkingLot in os.listdir(data_root):
      parkingLotPath = os.path.join(data_root, parkingLot)
      for statePath in os.listdir(parkingLotPath):
        img_filepath = os.path.join(parkingLotPath, statePath)
        for img in os.listdir(img_filepath):
          migPath = os.path.koin(img_filepath, img)
          self.imgNames.append(imgPath)
  
  def __len__(self):
    return len(self.imgNames)

  def __getitem__(self, idx):
    img = cv2.imread(self.imgNames[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(no.float32) / 255.0
    if 'busy' in self.imgNames[idx]:
      state = 0
    else:
      state = 1
    
    img_tensor = torch.from_numpy(img.transpose(2,0,1))
    return img_tensor, state

In [ ]:
# DIR_path = Path('./CNRPark-Patches-150x150/CNRPark-Patches-150x150/A/free')
# DIR_path = Path('./CNRPark-Patches-150x150/CNRPark-Patches-150x150/A/busy')
# DIR_path = Path('./CNRPark-Patches-150x150/CNRPark-Patches-150x150/B/free')
DIR_path = Path('./CNRPark-Patches-150x150/CNRPark-Patches-150x150/B/busy')
files_list = DIR_path.iterdir()

for f in files_list:
  img = cv2.imread(str(f))    # 문자로 읽어온다.

  if img is None:     # 사진이 없다면
    print('unable to open', f)
  else:               # 사진이 존재한다면
    h, w, c = img.shape     # 사진의 height, width, channel을 불러온다.
    if h != 150 or w != 150 or c != 3:    # 세개 조건중 하나라도 맞지 않는다면
      print('image', f, 'has wrong dimension : ', img.shape)
      resized = cv2.resize(img, dsize=(150, 150), interpolation=cv2.INTER_AREA)     # 사진 크기를 150 x 150으로 조정
      cv2.imwrite(str(f), resized)  # 다시 덮어 쓴다.

In [ ]:
# image_datasets = datasets.ImageFolder('./CNRPark-Patches-150x150/CNRPark-Patches-150x150/A',
#                                       transform = transforms.ToTensor())     # 이미지를 텐서로 변환
# image_datasets = datasets.ImageFolder('./CNRPark-Patches-150x150/CNRPark-Patches-150x150/B',
#                                       transform = transforms.ToTensor()
image_datasets = ImageMultiFolder('./CNRPark-Patches-150x150')

# 이미지를 보기 위한 코드
# dataloader = torch.utils.data.DataLoader(image_datasets, batch_size=1, shuffle=True, num_workers=8)

# for img, label in dataloader:   # 이미지를 dataloader로 가져오게 되면 이미지와 label을 같이 가지고 있음
#   imgToShow = img.numpy()   # tensor로 되어있으니까 numpy로 바꿔주는 작업
#   print(imgToShow.shape)
#   print(label)
#   imgToShow = np.transpose(imgToShow[0], (1,2,0))
#   # numpy에서 사용하는 이미지 배열과 tensor가 사용하는 순서(height, width, channels)가 다르기 때문에 transpose를 해주는 것
#   plt.imshow(imgToShow)
#   plt.show()


# train과 test 대이터 나누기

train_size = int(len(image_datasets)*0.8)
test_size = len(image_datasets) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(image_datasets, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size = BATCH_SIZE, 
                                           shuffle=True, 
                                           num_workers=8, 
                                           drop_last=True)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size = BATCH_SIZE, 
                                          shuffle=True, 
                                          num_workers=8, 
                                          drop_last=True)


AttributeError: ignored

### 학습 환경 설정
#### Loss: CrossEntropy
#### Optimizer : Adam

In [ ]:
device = torch.device('cuda:0' if torch.cuda. is_available() else 'cpu')    # 불러올 수 없는 환경이라면 cpu로
model = UltraSimpleParkingSpaceClassificationNet().to(device)
model = model.train()     # 학습모드
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

### 학습 시작

In [ ]:
num_epoch = 10

for i in range(num_epoch):
  for j, [image, label] in enumerate(train_loader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if j % 100 == 0:
      print('epoch:', i, ', batch:', j, ', loss is:', loss.data.item())

epoch: 0 , batch: 0 , loss is: 0.6875128746032715
epoch: 1 , batch: 0 , loss is: 0.6365188360214233
epoch: 2 , batch: 0 , loss is: 0.009958493523299694
epoch: 3 , batch: 0 , loss is: 0.014750929549336433
epoch: 4 , batch: 0 , loss is: 0.006372322794049978
epoch: 5 , batch: 0 , loss is: 0.0021463935263454914
epoch: 6 , batch: 0 , loss is: 0.000578028557356447
epoch: 7 , batch: 0 , loss is: 0.0003426806360948831
epoch: 8 , batch: 0 , loss is: 0.0011546829482540488
epoch: 9 , batch: 0 , loss is: 0.0004250503552611917


### 테스트 시작

In [ ]:
correct = 0
total = 0
model = model.eval()

with torch. no_grad():
   for image, label in test_loader:
     x = image.to(device)
     y_ = label.to(device)

     output = model.forward(x)
     _, output_index = torch.max(output, 1)   # index만 가지고 비교

     total += label.size(0)
     correct += (output_index == y_).sum().float()

print('Total: {}, Correct: {}'.format(total, correct))
print('Accuracy of Test Data: {}'.format(100 * correct / total))

Total: 1216, Correct: 1216.0
Accuracy of Test Data: 100.0
